# Azure SQL Database - User Role Assignment Auditing using Power Automate

## The goal of this exercise is to show how Microsoft Power Automate can be used to easily create an approval workflow for SQL role assignments.

### Anytime a user is assigned to a new role, a database trigger adds a record to the audit table we have created. The creation of this new record automatically kicks off a Power Automate workflow that sends an approval request to myself. Based on the approval/rejection, the record is updated with the appropriate notes and the role assignment either stays active or is reversed.

### Future ideas: This could be built out to require an approval before a role assignment is made to further secure the workflow. There could be additional parameterization added to dynamically select who needs to approve a request based on the company's reporting structure or the data owner.

In [1]:
-- Create a test login at the server level to play with
-- Will likely need to run the CREATE LOGIN script directly on master database outside of this notebook
USE master
CREATE LOGIN test_datascientist WITH password='Love2Audit!';

-- Associate a user with the login on the target database
USE AdventureWorks
CREATE USER test_datascientist FROM LOGIN test_datascientist;

-- Grant the user db_datareader role on AdventureWorks database
EXEC sp_addrolemember 'db_datawriter', 'test_datascientist';

-- Create a schema for the audit tables
-- I recommend using a specific auditing schema so you can make this schema read-only for all users
CREATE SCHEMA adt;

: Msg 40508, Level 16, State 1, Line 2
USE statement is not supported to switch between databases. Use a new connection to connect to a different database.

Total execution time: 00:00:00.038

In [21]:
-- Check and see which users have been assigned to which roles on the database
SELECT
    DP1.name AS DatabaseRoleName,   
    DP2.name AS UserName,
    DP2.[type_desc] AS UserType
FROM sys.database_role_members AS DRM  
RIGHT OUTER JOIN sys.database_principals AS DP1  
   ON DRM.role_principal_id = DP1.principal_id  
LEFT OUTER JOIN sys.database_principals AS DP2  
   ON DRM.member_principal_id = DP2.principal_id  
WHERE DP1.type = 'R'
    AND DP2.name IS NOT NULL
ORDER BY DP1.name;

(3 rows affected)

Total execution time: 00:00:00.052

DatabaseRoleName,UserName,UserType
db_datareader,test_datascientist,SQL_USER
db_owner,dbo,SQL_USER
db_owner,12308@lbmc.net,EXTERNAL_USER


In [5]:
-- Setup auditing tables
DROP TABLE IF EXISTS adt.UserAccessAudit

-- Create the adt.UserAccessAudit table to track audit results
CREATE TABLE [adt].[UserAccessAudit] (
    RecordNumber INT IDENTITY(1,1) PRIMARY KEY,
    ChangeTimestamp DATETIME NOT NULL,
    DatabaseRoleName VARCHAR(100) NOT NULL,
    UserName VARCHAR(100) NOT NULL,
    ChangedBy VARCHAR(100) NOT NULL,
    Authorized INT NULL,
    AuthorizedBy VARCHAR(100) NULL,
    AuthorizationDateTime DATETIME NULL,
    AuthorizationNote VARCHAR(MAX) NULL
)

Commands completed successfully.

Total execution time: 00:00:00.417

In [ ]:
-- Create a trigger on the database that fires anytime a user is adding to a role
-- Use this trigger to create a record in adt.UserAccessAudit

IF EXISTS (SELECT * FROM sys.triggers
    WHERE name = 'ddl_user_role_audit')
DROP TRIGGER ddl_user_role_audit
ON DATABASE;
GO

CREATE TRIGGER ddl_user_role_audit
ON DATABASE
FOR ADD_ROLE_MEMBER 
AS
BEGIN

    -- Get the EVENTDATE in XML format
    DECLARE @data XML
    SET @data = EVENTDATA()

    -- Parse out the tsql command
    DECLARE @tsql NVARCHAR(100)
    SET @tsql = @data.value('(/EVENT_INSTANCE/TSQLCommand)[1]', 'nvarchar(2000)')

    -- Find the character index values for the database role and user
    DECLARE @role_start_index INT
    DECLARE @role_end_index INT
    DECLARE @user_start_index INT
    DECLARE @user_end_index INT

    SET @role_start_index = CHARINDEX('[', @tsql, 1)
    SET @role_end_index = CHARINDEX(']', @tsql, 1)

    SET @user_start_index = CHARINDEX('[', @tsql, @role_start_index + 1)
    SET @user_end_index = CHARINDEX(']', @tsql, @role_end_index + 1)

    -- Parse out the database role and user
    DECLARE @database_role VARCHAR(MAX)
    DECLARE @username VARCHAR(MAX)

    SET @database_role = SUBSTRING(@tsql, @role_start_index + 1, @role_end_index - @role_start_index - 1)
    SET @username = SUBSTRING(@tsql, @user_start_index + 1, @user_end_index - @user_start_index - 1)

    -- Add a record to the audit table
    INSERT INTO [adt].[UserAccessAudit] (
        ChangeTimestamp
        ,DatabaseRoleName
        ,UserName
        ,ChangedBy
    )

    SELECT
    GETDATE()
    ,@database_role
    ,@username
    ,CONVERT(NVARCHAR(100), CURRENT_USER)
END


In [20]:
-- Test the trigger
-- Once the Power Automate flow is enabled this will also trigger the approval flow
EXEC sp_droprolemember 'db_datawriter', 'test_datascientist';
EXEC sp_addrolemember 'db_datawriter', 'test_datascientist';

SELECT * 
FROM adt.UserAccessAudit
WHERE RecordNumber = (SELECT MAX(RecordNumber) FROM adt.UserAccessAudit)

(1 row affected)

Total execution time: 00:00:00.069

RecordNumber,ChangeTimestamp,DatabaseRoleName,UserName,ChangedBy,Authorized,AuthorizedBy,AuthorizationDateTime,AuthorizationNote
16,2021-03-29 19:13:18.907,db_datawriter,test_datascientist,12308@lbmc.net,NULL,NULL,NULL,NULL


## Power Automate Approval Workflow

The Power Automate flow is automatically triggered anytime a new row is added to adt.UserAccessAudit, so the database trigger adding these records automatically kicks off the approval process.

If a role change is **approved**, then the approval is recorded on the row in adt.UserAccessAudit and no other actions are needed.

If a role change is **rejected**, then the rejection is recorded on the row in adt.UserAccessAudit and then the role assignment is dropped.